In [ ]:
from h5py import File
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import tqdm
from collections import Counter

#matplotlib.use("TkAgg")

In [ ]:
filename = "/Users/kartik/Dropbox/ips6/Data/test/data.raw.h5"

In [ ]:
def extract_spikes_from_h5(filename):
    try:
        f = File(filename, "r")
        recordings = f["data_store"]["data0000"]
        spikes = np.array(recordings["spikes"])

        channel = spikes["channel"]
        time = (spikes["frameno"] - (min(spikes["frameno"]))) / 20000
        duration = recordings["stop_time"][0] - recordings["start_time"][0]

        mapping = pd.DataFrame(f['/data_store/data0000/settings/mapping'][:])

        channel_times = {}
        for c, t in zip(channel, time):
            if c not in channel_times:
                channel_times[c] = []
            channel_times[c].append(t)
        raster = [channel_times[channel] for channel in sorted(channel_times)]
        return channel, time, raster, mapping

    except:
        print("Path does not contain an h5 file")
        return [], [], [], []


In [ ]:
channel, time, raster, mapping = extract_spikes_from_h5(filename)
spiketimes = pd.DataFrame([channel, time]).T
spiketimes.columns = ["channel","time"]

# Single Burst Only

In [ ]:
window = 0.002 #s
duration = 1.8 #s
start = 41.3

for e, i in tqdm.tqdm(enumerate(np.arange(start, start+duration, window)), total=int(duration/window)):
#for e,i in enumerate(np.arange(start, start+duration, window)):
    st_window = spiketimes[(spiketimes["time"] >= i) & (spiketimes["time"] < i+window)].reset_index(drop=True)
    st_window["channel"] = st_window["channel"].values.astype(int)
    st_window_mapping = pd.merge(st_window, mapping, on="channel", how="left")

    fr_mapping = st_window_mapping.groupby(["channel"]).agg(Count=('channel', 'size'),
                                                            X=('x', 'first'), Y=('y', 'first')).reset_index()

    all_spiketimes = spiketimes[(spiketimes["time"] >= start) & (spiketimes["time"] < i+window)]

    # Create a scatter plot as a heatmap
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(2, 3, width_ratios=[2, 2, 6], height_ratios=[1, 3])

    ax0 = fig.add_subplot(gs[0, :2])
    ax0.hist(all_spiketimes["time"], bins=np.arange(start, start+duration, window))
    ax0.set_xlim(start, start+duration)
    ax0.set_ylim(0, 16)

    window_raster_tmp = all_spiketimes.groupby('channel')['time'].apply(list).reset_index()
    window_raster = window_raster_tmp["time"].tolist()
    ax1 = fig.add_subplot(gs[1, :2])
    ax1.scatter(all_spiketimes["time"],all_spiketimes["channel"], s=2)
    ax1.set_xlim(start, start+duration)
    ax1.set_ylim(0, 1000)

    ax3 = fig.add_subplot(gs[1, 2])
    scatter = ax3.scatter(fr_mapping['X'], fr_mapping['Y'], c=fr_mapping['Count'], s=50,
                          cmap='plasma', alpha=0.6, edgecolors='w',vmin=0, vmax=3)
    cbar = plt.colorbar(scatter, ax=ax3, label='Count of Channels')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.xlim(min(mapping["x"].values), max(mapping["x"]))
    plt.ylim(min(mapping["y"].values), max(mapping["y"].values))
    plt.grid(True)  # Optional: adds a grid for better visualization of the scale
    plt.savefig("gif/"+str(e)+".png")
    plt.close()

# Full Recording

In [ ]:
window = 1 #s
duration = 300 #s
start = 0

for e, i in tqdm.tqdm(enumerate(np.arange(start, start+duration, window)), total=int(duration/window)):
#for e,i in enumerate(np.arange(start, start+duration, window)):
    st_window = spiketimes[(spiketimes["time"] >= i) & (spiketimes["time"] < i+window)].reset_index(drop=True)
    st_window["channel"] = st_window["channel"].values.astype(int)
    st_window_mapping = pd.merge(st_window, mapping, on="channel", how="left")

    fr_mapping = st_window_mapping.groupby(["channel"]).agg(Count=('channel', 'size'),
                                                            X=('x', 'first'), Y=('y', 'first')).reset_index()

    all_spiketimes = spiketimes[(spiketimes["time"] >= start) & (spiketimes["time"] < i+window)]

    # Create a scatter plot as a heatmap
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(2, 3, width_ratios=[2, 2, 6], height_ratios=[1, 3])

    ax0 = fig.add_subplot(gs[0, :2])
    ax0.hist(all_spiketimes["time"], bins=np.arange(start, start+duration, window))
    ax0.set_xlim(start, start+duration)
    ax0.set_ylim(0, 1500)

    window_raster_tmp = all_spiketimes.groupby('channel')['time'].apply(list).reset_index()
    window_raster = window_raster_tmp["time"].tolist()
    ax1 = fig.add_subplot(gs[1, :2])
    ax1.scatter(all_spiketimes["time"],all_spiketimes["channel"], s=0.2, alpha=0.5)
    ax1.set_xlim(start, start+duration)
    ax1.set_ylim(0, 1000)

    ax3 = fig.add_subplot(gs[1, 2])
    scatter = ax3.scatter(fr_mapping['X'], fr_mapping['Y'], c=fr_mapping['Count'], s=fr_mapping['Count']*25,
                          cmap='plasma', alpha=0.6, edgecolors='w',vmin=0, vmax=10)
    cbar = plt.colorbar(scatter, ax=ax3, label='Count of Channels')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.xlim(min(mapping["x"].values), max(mapping["x"]))
    plt.ylim(min(mapping["y"].values), max(mapping["y"].values))
    plt.grid(True)  # Optional: adds a grid for better visualization of the scale
    plt.savefig("gif_full_recording/"+str(e)+".png")
    plt.close()

### After I converted the PNGs to a GIF. You can probably make a movie (ffempg) instead of this way.

![Full Recording](/gif_full_recording/full_recording.gif "Full Recording")

# Single burst only + consecutive burst counter

In [ ]:
# Initialize the channel frequency counter
channel_frequency = Counter()

# Example data loading steps (make sure to define or load 'spiketimes' and 'mapping')
window = 0.002 # seconds
duration = 1.8 # seconds
start = 41.3

for e, i in tqdm.tqdm(enumerate(np.arange(start, start + duration, window)), total=int(duration/window)):
    st_window = spiketimes[(spiketimes["time"] >= i) & (spiketimes["time"] < i + window)].reset_index(drop=True)
    st_window["channel"] = st_window["channel"].values.astype(int)
    st_window_mapping = pd.merge(st_window, mapping, on="channel", how="left")

    fr_mapping = st_window_mapping.groupby(["channel"]).agg(Count=('channel', 'size'),
                                                            X=('x', 'first'), Y=('y', 'first')).reset_index()

    all_spiketimes = spiketimes[(spiketimes["time"] >= start) & (spiketimes["time"] < i+window)]

    # Update channel frequency
    for channel in fr_mapping["channel"]:
        channel_frequency[channel] += 1

    # Calculate point sizes based on frequency
    fr_mapping['Size'] = fr_mapping['channel'].apply(lambda x: channel_frequency[x] * 10)  # Adjust size factor as needed

    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(2, 3, width_ratios=[2, 2, 6], height_ratios=[1, 3])

    ax0 = fig.add_subplot(gs[0, :2])
    ax0.hist(all_spiketimes["time"], bins=np.arange(start, start+duration, window))
    ax0.set_xlim(start, start+duration)
    ax0.set_ylim(0, 16)

    window_raster_tmp = all_spiketimes.groupby('channel')['time'].apply(list).reset_index()
    window_raster = window_raster_tmp["time"].tolist()
    ax1 = fig.add_subplot(gs[1, :2])
    ax1.scatter(all_spiketimes["time"],all_spiketimes["channel"], s=2)
    ax1.set_xlim(start, start+duration)
    ax1.set_ylim(0, 1000)

    ax3 = fig.add_subplot(gs[1, 2])
    scatter = ax3.scatter(fr_mapping['X'], fr_mapping['Y'], c=fr_mapping['Count'], s=fr_mapping['Size'],
                          cmap='plasma', alpha=0.6, edgecolors='w', vmin=0, vmax=3)
    cbar = plt.colorbar(scatter, ax=ax3, label='Count of Channels')
    ax3.set_xlabel('X Coordinate')
    ax3.set_ylabel('Y Coordinate')
    ax3.set_xlim(min(mapping["x"])-25, max(mapping["x"])+25)
    ax3.set_ylim(min(mapping["y"])-25, max(mapping["y"])+25)
    plt.grid(True)
    plt.savefig("gif/" + str(e) + ".png")
    plt.close()

![High res network burst frequency](/gif/high_res_frequency.gif)